## Title :
RNN from scratch

## Description :
The aim of this exercise is to understand what happens within an RNN unit that is wrapped within the tensorflow.keras.layers.SimpleRNN

The idea is to write a Recurrent Neural Network from scratch that generates names of dinosaurs by training on the existing names character-wise.

## Instructions:
- Read the file `dino.txt` and convert all the names in the files to lowercase.
- Save the names in the file as a list.
- Get the number of words in the file and the vocabulary size which is equal to the number of alphabets plus the newline character.
- Define a dictionary `char_to_ix`  where the key is the sorted vocabulary and the value is the integer value assigned to it.
- Define a dictionary `ix_to_char`  where the key is the integer value assigned to the unique vocabulary and the value is the sorted vocabulary.
- To get the model parameters (weights and biases) call the get_weights function twice once by:
    - Setting the `random` parameter as 1 to get random weights
    - Setting the `random` parameter as 0 to get the trained weights by specifying the number of iterations.
- Define a function `rnn_model` that takes in the network parameters and outputs the generated dinosaur name based on the instructions in the scaffold.

## Hints:

$$h_t\ =\ \tanh\left(\ Uh_{t-1}\ +\ Vx_t\ +\ \beta_1\right)$$

$$Y_{t\ }\ =\ \sigma\left(Wh_t\ +\ \beta_2\right)$$


<a href="https://docs.python.org/3/library/functions.html#sorted" target="_blank">sorted()</a> Return a new sorted list from the items in iterable.

<a href="https://book.pythontips.com/en/latest/enumerate.html" target="_blank">enumerate()</a> Allows to loop over something and have an automatic counter.

<a href="https://docs.python.org/3/library/stdtypes.html?highlight=lower#str.lower" target="_blank">lower()</a> Return a copy of the string with all the cased characters converted to lowercase.

<a href="https://docs.python.org/3/library/stdtypes.html?highlight=strip#str.strip" target="_blank">strip()</a> Return a copy of the string with the leading and trailing characters removed.

<a href="https://numpy.org/doc/stable/reference/random/generated/numpy.random.shuffle.html" target="_blank">np.random.shuffle()</a> Modify a sequence in-place by shuffling its contents.

<a href="https://numpy.org/doc/stable/reference/generated/numpy.zeros.html?highlight=zeros#numpy.zeros" target="_blank">np.zeros()</a> Return a new array of given shape and type, filled with zeros.

<a href="https://docs.python.org/3/library/stdtypes.html?highlight=join#str.join" target="_blank">join()</a> Return a string which is the concatenation of the strings in iterable.

<a href="https://numpy.org/doc/stable/reference/generated/numpy.tanh.html?highlight=tanh#numpy.tanh" target="_blank">np.tanh()</a> Compute hyperbolic tangent element-wise.

<a href="https://numpy.org/doc/stable/reference/generated/numpy.dot.html?highlight=dot#numpy.dot" target="_blank">np.dot()</a> Returns the dot product of two arrays.

In [4]:
# Import necessary libraries
import random
import numpy as np
from helper import softmax, get_weights


In [30]:
# Function to predict the next set of characters which forms the dinosaur name
def rnn_model(parameters, char_to_ix):

    # Get the weights and biases from the parameters dictionary
    U, V, W = parameters['U'], parameters['V'], parameters['W']
    beta1, beta2 = parameters['beta1'], parameters['beta2']

    # Get the size of the vocabulary i.e. 27
    # One for each alphabet plus the new line character
    vocab_size = beta2.shape[0]

    # Get the size of the weights
    n_h = U.shape[1]    

    # Initialize the input as an array of zeroes with size as (vocab_size,1)
    # This one-hot encodes the input
    x = np.zeros((vocab_size,1))

    # Initialize the inital hidden state as an array of zeroes with size as (n_h,1)    
    h_prev = np.zeros((n_h, 1))

    # Initialize a list to store the indices of the predicted characters
    indices = []
    
    # Initialize an idx variable to hold the index values of the characters 
    idx = -1 
    
    # Initialize a counter to fix the maximum length of the predicted word
    counter = 0

    # Get the value of the new line from the char_to_ix dictionary
    newline_character = char_to_ix['\n']
    
    # Loop until the newline_character is predicted or until the max length of the word is 50
    while (idx != newline_character and counter != 50):

        # Compute the new state h of the RNN unit using the equation given in the instructions
        # WRONG h = np.tanh(U * h_prev + V * x + beta1)
        h = np.tanh(np.dot(U,h_prev) + np.dot(V, x) + beta1)

        # Compute the output of the RNN unit using the equation 
        # given in the instructions using the softmax function
        y = softmax(np.dot(W,h) + beta2)

        # Get the index value of the predicted/generated character
        # Instead of taking the argmax, we perform sampling on the probabilities 
        # got from the softmax function
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())

        # Append the index value to the indices list
        indices.append(idx)
        
        # Initialize an array of with zeroes with size (vocab_size,1)
        x = np.zeros((vocab_size, 1))

        # Set the idx position of x as 1.
        # This will act as the output y and the next input. 
        x[idx] = 1
        
        # Update the previous state value with the current state
        h_prev = h
        
        # Increment the counter
        counter+=1
    
    # If the counter value reaches 50 append a newline character to the indices list
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    # Return the list of indices
    return indices



In [7]:
sorted(["b", "c", "a"])

['a', 'b', 'c']

In [8]:
a = ["abs", "bs", "cs"]
np.random.shuffle(a)
a

['abs', 'cs', 'bs']

In [36]:
# Read the dinos.txt file
data = open('../data/dinos.txt', 'r').read()

# Convert the data to lower case
data= data.lower()

# Convert the file data into list
chars = list(set(data))

# Get length of the file and length of the vocabulary
data_size, vocab_size = len(data), len(chars)

# Define a dictionary with the sorted vocabulary as key and 
# value as a unique integer using enumerate
char_to_ix = {a: b for b, a in enumerate(sorted(chars))}

# Define a dictionary with the unique integers assigned to the sorted vocabulary as key
# and value as a unique integer using enumerate
ix_to_char = {a:b for a, b in enumerate(sorted(chars))}

# Call the get_weights function to get the model weights
# To get random weights set random=1
# To get the trained weights specify the number of iterations and set random=0
parameters = get_weights(num_iterations=20000, random=0)


In [25]:
U, V, W = parameters['U'], parameters['V'], parameters['W']
print(U.shape, V.shape, W.shape)

(50, 50) (50, 27) (27, 50)


In [29]:
beta1, beta2 = parameters['beta1'], parameters['beta2']
print(beta1.shape, beta2.shape)

(50, 1) (27, 1)


In [49]:
# Call the predict function defined above passing 
# the parameters dictionary, char_to_ix dictionary
sampled_indices = rnn_model(parameters, char_to_ix)

# Convert the list of indices returned by the predict function to 
# their respective characters and then join to form a word
txt = "".join([ix_to_char[index] for index in sampled_indices])

# Captializing the first character
txt = txt[0].upper() + txt[1:]  

# Print the generated dinosaur name 
print('%s' % (txt, ), end='')

Ariilosaurus


### ⏸ What do you observe from the generated names when the number of iterations in the get_weights function increase to 20,000 with random=0?

#### A. The length of the names generated increases proportionately with the number of iterations.
#### B. Insufficient storage because of large window size.
#### C. The names generated are better due to longer training.
#### D. Larger number of iterations causes the loss of the model to increase. 

In [ ]:
### edTest(test_chow1) ###
# Submit an answer choice as a string below (eg. if you choose option A, put 'A')
answer1 = 'C'

### ⏸ What is the difference in the generated name from when the model is given random weights and trained weights?  


In [ ]:
### edTest(test_chow2) ###
# Type your answer within in the quotes given
answer2 = '___'

In [10]:
with open("../data/dinos.txt") as f:
       examples = f.readlines()

examples = [x.lower().strip() for x in examples]

np.random.shuffle(examples)

for j in range(10):
    index = j % len(examples)
    X = [None] + [char_to_ix[ch] for ch in examples[index]]
    Y = X[1:] + [char_to_ix["\n"]]
    print(X)
    print(Y)

[None, 19, 11, 5, 12, 5, 20, 15, 14]
[19, 11, 5, 12, 5, 20, 15, 14, 0]
[None, 6, 21, 20, 1, 2, 1, 19, 1, 21, 18, 21, 19]
[6, 21, 20, 1, 2, 1, 19, 1, 21, 18, 21, 19, 0]
[None, 16, 12, 1, 20, 25, 3, 5, 18, 1, 20, 15, 16, 19]
[16, 12, 1, 20, 25, 3, 5, 18, 1, 20, 15, 16, 19, 0]
[None, 1, 14, 20, 1, 18, 3, 20, 15, 19, 1, 21, 18, 21, 19]
[1, 14, 20, 1, 18, 3, 20, 15, 19, 1, 21, 18, 21, 19, 0]
[None, 15, 19, 13, 1, 11, 1, 19, 1, 21, 18, 21, 19]
[15, 19, 13, 1, 11, 1, 19, 1, 21, 18, 21, 19, 0]
[None, 5, 18, 11, 5, 20, 21]
[5, 18, 11, 5, 20, 21, 0]
[None, 1, 18, 3, 15, 22, 5, 14, 1, 20, 15, 18]
[1, 18, 3, 15, 22, 5, 14, 1, 20, 15, 18, 0]
[None, 7, 1, 19, 15, 19, 1, 21, 18, 21, 19]
[7, 1, 19, 15, 19, 1, 21, 18, 21, 19, 0]
[None, 20, 1, 16, 21, 9, 1, 19, 1, 21, 18, 21, 19]
[20, 1, 16, 21, 9, 1, 19, 1, 21, 18, 21, 19, 0]
[None, 14, 5, 13, 5, 7, 20, 9, 1]
[14, 5, 13, 5, 7, 20, 9, 1, 0]


In [11]:
h0 = np.zeros((10, 1))

x, h, y_hat = {}, {}, {}
h[-1] = np.copy(h0)
loss = 0
for t in range(len(X)):
    x[t] = np.zeros((vocab_size, 1))
    if (X[t] != None):
        x[t][X[t]] = 1
        print(x[t])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.

In [14]:
dx

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [-1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [12]:
for i in reversed(range(10)):
    print(i)

9
8
7
6
5
4
3
2
1
0
